In [ ]:

!pip install surprise

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split
import random

In [ ]:
# завантаження датасету
data = Dataset.load_builtin('ml-100k')

# Отримання raw ratings
raw_ratings = data.raw_ratings

# вивантаження у форматі pd DataFrame для перегляду

df = pd.DataFrame(raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])

df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
# визначення сітки параметрів. Ці параметри ми обираємо самостійно.Можна змінювати для найкращих результатів.
param_grid = {
    'n_epochs': [12, 20, 24],
    'lr_all': [0.01, 0.03, 0.05],
    'reg_all': [0.02, 0.1, 0.2]
}

# використання GridSearchCV у підборі параметрів
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# вивід найкращих результатів
print("Best RMSE score:", gs.best_score['rmse'])
print("Best parameters:", gs.best_params['rmse'])

Best RMSE score: 0.9244698948579823
Best parameters: {'n_epochs': 12, 'lr_all': 0.03, 'reg_all': 0.1}


In [ ]:
# ініціалізація алгоритма
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'],
           lr_all=gs.best_params['rmse']['lr_all'],
           reg_all=gs.best_params['rmse']['reg_all'])

# оцінка моделі з використанням крос-валідації.cv = 3 - це кількість розбіттів для крос-валідації. Обираємо самостійно.
results = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
print("\nРезультати з найкращими параметрами:")
print("RMSE: ", np.mean(results['test_rmse']))
print("MAE: ", np.mean(results['test_mae']))

Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9291  0.9231  0.9202  0.9241  0.0037  
MAE (testset)     0.7349  0.7305  0.7267  0.7307  0.0034  
Fit time          0.76    0.77    0.75    0.76    0.01    
Test time         0.22    0.26    0.29    0.26    0.03    

Результати з найкращими параметрами:
RMSE:  0.9241421491595067
MAE:  0.7307220900896185


In [ ]:
#Аналогічні розрахунки виконаємо для NMF
from surprise import NMF

# Визначення сітки параметрів для NMF
param_grid = {
    'n_epochs': [50, 100],
    'lr_bu': [0.002, 0.005],
    'lr_bi': [0.002, 0.005],
    'reg_pu': [0.06, 0.1],
    'reg_qi': [0.06, 0.1]
}

# Використання GridSearchCV для підбору параметрів
gs = GridSearchCV(NMF, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)




In [ ]:
# Вивід найкращих результатів
print("Best RMSE score for NMF:", gs.best_score['rmse'])
print("Best parameters for NMF:", gs.best_params['rmse'])

# Ініціалізація алгоритму з найкращими параметрами
best_params = gs.best_params['rmse']
algo_nmf = NMF(n_epochs=best_params['n_epochs'],
               lr_bu=best_params['lr_bu'],
               lr_bi=best_params['lr_bi'],
               reg_pu=best_params['reg_pu'],
               reg_qi=best_params['reg_qi'])

# Оцінка моделі з використанням крос-валідації
results_nmf = cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Вивід результатів
print("\nРезультати з найкращими параметрами для NMF:")
print("RMSE: ", np.mean(results_nmf['test_rmse']))
print("MAE: ", np.mean(results_nmf['test_mae']))

Best RMSE score for NMF: 0.9480610662821926
Best parameters for NMF: {'n_epochs': 100, 'lr_bu': 0.002, 'lr_bi': 0.002, 'reg_pu': 0.1, 'reg_qi': 0.1}
Evaluating RMSE, MAE of algorithm NMF on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9483  0.9430  0.9578  0.9497  0.0061  
MAE (testset)     0.7494  0.7486  0.7608  0.7529  0.0056  
Fit time          3.71    3.89    3.47    3.69    0.17    
Test time         0.40    0.16    0.17    0.24    0.11    

Результати з найкращими параметрами для NMF:
RMSE:  0.9497000579035567
MAE:  0.752923259494313


SVD++

In [ ]:
from surprise import SVDpp
from surprise.model_selection import GridSearchCV, cross_validate
import numpy as np

# Визначення сітки параметрів для SVD++
param_grid = {
    'n_epochs': [20, 50],
    'lr_all': [0.005, 0.01],
    'reg_all': [0.02, 0.1]
}

# Використання GridSearchCV для підбору параметрів
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Вивід найкращих результатів
print("Best RMSE score for SVD++:", gs.best_score['rmse'])
print("Best parameters for SVD++:", gs.best_params['rmse'])


Best RMSE score for SVD++: 0.9225424141194409
Best parameters for SVD++: {'n_epochs': 50, 'lr_all': 0.01, 'reg_all': 0.1}


In [ ]:
# Ініціалізація алгоритму з найкращими параметрами
best_params = gs.best_params['rmse']
algo_svdpp = SVDpp(n_epochs=best_params['n_epochs'],
                   lr_all=best_params['lr_all'],
                   reg_all=best_params['reg_all'])

# Оцінка моделі з використанням крос-валідації
results_svdpp = cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

# Вивід результатів
print("\nРезультати з найкращими параметрами для SVD++:")
print("RMSE: ", np.mean(results_svdpp['test_rmse']))
print("MAE: ", np.mean(results_svdpp['test_mae']))

Evaluating RMSE, MAE of algorithm SVDpp on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9168  0.9232  0.9224  0.9208  0.0028  
MAE (testset)     0.7228  0.7277  0.7272  0.7259  0.0022  
Fit time          47.22   47.65   46.27   47.05   0.58    
Test time         6.50    7.18    7.32    7.00    0.36    

Результати з найкращими параметрами для SVD++:
RMSE:  0.9208221054955467
MAE:  0.7258760030065354


Значення RMSE вказує на середнє квадратичне відхилення між фактичними та передбаченими рейтингами, тобто чим менше значення RMSE, тим краща модель.
Fold 1, Fold 2, Fold 3: Це результати моделі на трьох різних частинах даних, використаних у крос-валідації. Крос-валідація поділяє дані на кілька частин (у цьому випадку 3) і перевіряє модель на кожній частині, щоб отримати середнє значення та стандартне відхилення помилок.
